# ESCUELA COLOMBIANA DE INGENIERÍA:
# PRINCIPIOS Y TECNOLOGÍAS IA 2025-2
## CLASIFICACIÓN INTELIGENTE DE RESIDUOS SÓLIDOS MEDIANTE REDES NEURONALES CONVOLUCIONALES
## PROYECTO

**OBJETIVO GENERAL**

Desarrollar un agente supervisado de clasificación automática de residuos sólidos mediante una red neuronal convolucional (CNN) que permita diferenciar materiales orgánicos, inorgánicos y aprovehcables, contribuyendo a la gestión ambiental sostenible en la ciudad de Bogotá.

**OBJETIVOS**

1. Diseña red convolucional (CNN).
2. Implementar red convolucional (CNN).
3. Evaluar su desempeño mediante métricas apropiadas.


In [ ]:
# ---
%pip install -q scikit-learn matplotlib seaborn

%pip install kaggle

%pip install --quiet kagglehub

In [ ]:
import kagglehub
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import itertools

KAGGLE_DATASET_ID = "phenomsg/waste-classification"
path_to_downloaded_dataset = kagglehub.dataset_download(KAGGLE_DATASET_ID)
print("Path to downloaded dataset files:", path_to_downloaded_dataset)

DATA_DIR = path_to_downloaded_dataset

print(f"Inspecting contents of DATA_DIR: {DATA_DIR}")
image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
found_non_image_files = []
all_image_paths = []
all_image_labels = []

class_names = sorted([d.name for d in os.scandir(DATA_DIR) if d.is_dir()])
class_to_idx = {name: idx for idx, name in enumerate(class_names)}

for root, _, files in os.walk(DATA_DIR):
    relative_path = os.path.relpath(root, DATA_DIR)
    class_name = None
    if relative_path != '.':
        parts = relative_path.split(os.sep)
        if len(parts) >= 1:
            class_name = parts[0]

    if class_name not in class_names:
        continue

    for file in files:
        file_path = os.path.join(root, file)

        if os.path.getsize(file_path) == 0:
            found_non_image_files.append(file_path)
            continue

        ext = os.path.splitext(file_path)[1].lower()
        if ext not in image_extensions:
            found_non_image_files.append(file_path)
        else:
            all_image_paths.append(file_path)
            all_image_labels.append(class_to_idx[class_name])

if found_non_image_files:
    print(f"Total non-image/corrupted files found: {len(found_non_image_files)}.")
else:
    print("No unusual files found.")

NUM_CLASSES = len(class_names)
print("Clases:", class_names)
print(f"Found {len(all_image_paths)} valid images.")

SEED = 42
combined = list(zip(all_image_paths, all_image_labels))
np.random.seed(SEED)
np.random.shuffle(combined)
all_image_paths, all_image_labels = zip(*combined)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

TRAIN_SPLIT_RATIO = 0.8
VAL_SPLIT_RATIO = 0.1
TEST_SPLIT_RATIO = 0.1

total_images = len(all_image_paths)
train_size = int(TRAIN_SPLIT_RATIO * total_images)
val_size = int(VAL_SPLIT_RATIO * total_images)
test_size = total_images - train_size - val_size

train_paths = all_image_paths[:train_size]
train_labels = all_image_labels[:train_size]

val_paths = all_image_paths[train_size : train_size + val_size]
val_labels = all_image_labels[train_size : train_size + val_size]

test_paths = all_image_paths[train_size + val_size : train_size + val_size + test_size]
test_labels = all_image_labels[train_size + val_size : train_size + val_size + test_size]

print(f"Dataset sizes: Train={len(train_paths)}, Val={len(val_paths)}, Test={len(test_paths)}")

def _decode_and_resize(image_path_tensor):
    image_path_str = image_path_tensor.numpy().decode('utf-8')
    try:
        img_raw = tf.io.read_file(image_path_str)
        img = tf.image.decode_image(img_raw, channels=3, expand_animations=False)
        if img.shape.rank is None or img.shape.rank < 3:
            raise ValueError()
        img = tf.image.resize(img, IMG_SIZE)
        img = tf.cast(img, tf.float32)
        return img
    except:
        return tf.zeros(IMG_SIZE + (3,), dtype=tf.float32)

def preprocess_image(image_path, label):
    img = tf.py_function(func=_decode_and_resize, inp=[image_path], Tout=tf.float32)
    img.set_shape(IMG_SIZE + (3,))
    label = tf.one_hot(label, NUM_CLASSES)
    return img, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((list(train_paths), list(train_labels))) \
             .map(preprocess_image, num_parallel_calls=AUTOTUNE) \
             .shuffle(1000) \
             .batch(BATCH_SIZE) \
             .prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((list(val_paths), list(val_labels))) \
           .map(preprocess_image, num_parallel_calls=AUTOTUNE) \
           .batch(BATCH_SIZE) \
           .prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((list(test_paths), list(test_labels))) \
            .map(preprocess_image, num_parallel_calls=AUTOTUNE) \
            .batch(BATCH_SIZE) \
            .prefetch(AUTOTUNE)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.06),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

checkpoint_cb = keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss")
earlystop_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)
reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)
tensorboard_cb = keras.callbacks.TensorBoard(log_dir="./logs")

history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb, tensorboard_cb]
)

base_model.trainable = True
fine_tune_at = 50
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb, tensorboard_cb]
)

model.load_weights("best_model.h5")
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc, "Test loss:", test_loss)

y_true_list = []
for images, labels in test_ds:
    y_true_list.append(labels.numpy())
y_true = np.concatenate(y_true_list, axis=0)
y_true = np.argmax(y_true, axis=1)

preds = model.predict(test_ds)
y_pred = np.argmax(preds, axis=1)

print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)

def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.tight_layout()

plot_confusion_matrix(cm, class_names)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

def predict_url(image_url):
    try:
        path = tf.keras.utils.get_file(origin=image_url)
        
        img = keras.utils.load_img(path, target_size=(500, 500))
        plt.imshow(img)
        plt.axis('off')
        plt.show()
        img_array = keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) 

        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        
        predicted_class = class_names[np.argmax(score)]
        confidence = 100 * np.max(score)
        
        print(f" Predicción: {predicted_class}")
        print(f" Confianza: {confidence:.2f}%")
        
        print("\n--- Probabilidades por clase ---")
        for i, name in enumerate(class_names):
            print(f"{name}: {100 * predictions[0][i]:.2f}%")
            
    except Exception as e:
        print(f" Error al cargar la imagen: {e}")


url = "" 

predict_url(url)